In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly

In [2]:
buyer = pd.read_excel('../info/buyer.xlsx')
buyer.columns = ['buyer_id', 'address']

In [3]:
seller = pd.read_excel('../info/seller.xlsx')
seller.columns = ['company_id', 'postcode', 'city']

In [4]:
# 328 дупликатов адреса
buyer_address = pd.DataFrame(buyer.address.value_counts()).reset_index()
buyer_address_iter = iter(buyer_address.address.values)
"{}/{} duplicates. Percentage of duplicates: {}. Number of unique companies: {}".format(dup:=buyer_address['count'].apply(lambda x: x - 1 if x > 1 else 0).sum(), buyer.shape[0], f'{dup/buyer.shape[0]*100:.2f}%', buyer_address.shape[0])

'193/1856 duplicates. Percentage of duplicates: 10.40%. Number of unique companies: 1663'

In [67]:
next(buyer_address_iter)

'191014, г. Санкт-Петербург, ул. Парадная, д. 3, корп. 1 '

# Class with implementation of obtaining coordinates by address-line and calculating the distance between two geo-points

In [133]:
import osrm
from googlesearch import search
import requests
from fake_headers import Headers
from bs4 import BeautifulSoup
from selenium import webdriver as wd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pathlib import Path
from tqdm import tqdm
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import random
import time
from pathlib import Path
import shutil
from copy import deepcopy



coordinates = [[55.7522, 37.6156], [55.7887, 49.1221], [55.3949, 43.8399]]
tgt_coord = [55.7254, 52.4112]


class GeoProcessor:
    def __init__(self):
        self.osrm_client = osrm.Client(host='http://router.project-osrm.org')
        self.headers = Headers(
                os="mac",  # Generate ony Windows platform
                headers=True  # generate misc headers
        ).generate()
        
        self.options = wd.ChromeOptions()
        self.options.add_argument("--enable-javascript")
        self.options.add_argument("--disable-blink-features=AutomationControlled")  # Скрываем признак автоматизации
        self.options.add_argument("--disable-infobars")  # Отключаем инфо-бар Selenium
        self.options.add_argument("--start-maximized")  # Полноэкранный режим для имитации реального пользователя
        self.options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36")  # Задаем реальный User-Agent

        # Отключаем дополнительные всплывающие окна и уведомления
        prefs = {"profile.default_content_setting_values.notifications": 2, 
                "profile.default_content_setting_values.geolocation": 2}
        self.options.add_experimental_option("prefs", prefs)

    def distance(self, from_coords, tgt_coords):
        """ (широта, долгота) """
        coordinates_osrm = [from_coords[::-1], tgt_coords[::-1]] # lat, lon
        osrm_response = self.osrm_client.route(coordinates=coordinates_osrm, overview=osrm.overview.full)
        dist_osrm = osrm_response.get('routes')[0].get('distance')/1000 # in km
        print('distance using OSRM: ', dist_osrm)
        return dist_osrm
    
    def google_address(self, addr: str, num_results=5, proxy=None, banned=False, captcha_sleep=2):
        if banned:
            res = list(self.search_selenium(addr, num_results=num_results, captcha_sleep=captcha_sleep))
        else:
            res = list(search(addr, sleep_interval=1, num_results=num_results, proxy=proxy))
        return res
    

    def search_selenium(self, address: str, num_results=5, captcha_sleep=2):
        options = deepcopy(geopc.options)
        options.page_load_strategy = 'eager'

        browser = wd.Chrome(options=options)
        browser.get('https://www.google.com/search')
        textarea = browser.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea")
        textarea.send_keys(address)
        textarea.send_keys(Keys.ENTER)
        time.sleep(captcha_sleep)
        html = browser.page_source
        soup = BeautifulSoup(html, "html.parser")
        return self.parse_soup(soup, num_results=num_results)
            
    def parse_soup(self, soup, num_results=5):
        fetched_results = 0
        while fetched_results < num_results:
            result_block = soup.find_all("div", attrs={"class": "g"})
            new_results = 0  # Keep track of new results in this iteration

            for result in result_block:
                # Find link, title, description
                link = result.find("a", href=True)
                title = result.find("h3")
                description_box = result.find("div", {"style": "-webkit-line-clamp:2"})

                if link and title and description_box:
                    description = description_box.text
                    fetched_results += 1
                    new_results += 1
                    # if advanced:
                        # yield SearchResult(link["href"], title.text, description)
                    # else:
                    yield link["href"]

                if fetched_results >= num_results:
                    break  # Stop if we have fetched the desired number of results

            if new_results == 0:
                break


    def parse_freicon(self, freicon_url: str='https://online.freicon.ru/info/stations/799101'):
        """ If alta not in urls """
        try:
            browser = wd.Chrome(options=self.options)
            browser.get(freicon_url)
            lat = float(browser.find_element(By.XPATH, '/html/body/app-root/app-admin-layout/div/main/app-info/app-station-details/section/div[1]/div[13]/div[1]/span').text)
            lon = float(browser.find_element(By.XPATH, '/html/body/app-root/app-admin-layout/div/main/app-info/app-station-details/section/div[1]/div[13]/div[2]/span').text)
            return lat, lon
        except:
            return None
    
    def parse_alta(self, alta_url: str='https://www.alta.ru/railway/station/81200/'):
        """ More prioitised than freicon.ru"""
        try:
            soup = BeautifulSoup(requests.get(alta_url, headers=self.headers).text, 'html.parser')
            lon = float(soup.find('div', class_='dib pl-10').text.split(':')[1].strip())
            lat = float(soup.find('div', class_='dib pr-10').text.split(':')[1].strip())
            lat, lon
            return lat, lon
        except:
            return None
    
    def parse_yandex(self, ya_url: str='https://yandex.ru/maps/970/novorossiysk/house/krasnaya_ulitsa_108/Z04YcQVgSUYDQFpufXt2dHlhZA==/', soup=None):
        """ Best if object is not a railway station """
        try:
            if soup is None:
                soup = BeautifulSoup(requests.get(ya_url, headers=self.headers).text, 'html.parser')
            lat, lon = map(float, map(str.strip, soup.find('div', class_='toponym-card-title-view__coords-badge').text.split(',')))
            return lat, lon
        except:
            return None
        
    def parse_yandex_final(self, ya_url: str='https://yandex.ru/maps/org/otdeleniye_pochtovoy_svyazi_644040/170954544743/'):
        try:
            browser = wd.Chrome(options=self.options)
            browser.get(ya_url)

            num_try = 0
            while ('?ll=' not in self.basename) and (num_try < 5):
                time.sleep(0.2)
                self.basename = Path(browser.current_url).name
                num_try += 1
            lon = self.basename.split('ll=')[1].split('%2C')[0]
            lat = self.basename.split('%2C')[1].split('&')[0]
            try:
                lon = float(lon)
                lat = float(lat)
            except:
                pass
            return lat, lon
        except:
            return None
    
    def parse_2gis(self, gis_url: str='https://2gis.ru/noyabrsk/search/%D0%A2%D1%80%D1%83%D0%B1%D0%BD%D0%B0%D1%8F%20%D0%B1%D0%B0%D0%B7%D0%B0'):
        """
            https://2gis.ru/krasnodar/directions/points/%7C39.086777%2C45.043662%3B3237597887345181

            template: %7C{lon}%2C{lat}%{hash}'
            swap => lat, lon
        """
        try:
            browser = wd.Chrome(options=self.options)
            browser.get(gis_url)
            try:
                elem = browser.find_element(By.XPATH, '/html/body/div[2]/div/div/div[1]/div[1]/div[3]/div/div/div[2]/div/div/div[2]/div[2]/div/div[1]/div/div/div/div/div[1]/div[2]/div[3]/a')
            except:
                browser.find_element(By.XPATH, '/html/body/div[2]/div/div/div[1]/div[1]/div[3]/div/div/div[2]/div/div/div/div[2]/div[2]/div[1]/div/div/div/div[2]/div/div/div').click()
                elem = browser.find_element(By.XPATH, '/html/body/div[2]/div/div/div[1]/div[1]/div[3]/div[2]/div/div/div/div/div[2]/div[2]/div/div[1]/div/div/div/div/div[1]/div[1]/div[3]/a')
            elem.click()
            self.basename = Path(browser.current_url).name
            while '%7C' not in self.basename:
                time.sleep(0.2)
                self.basename = Path(browser.current_url).name
            lon = float(self.basename.split('%7C')[1].split('%2C')[0])
            lat = float(self.basename.split('%2C')[1].split('%')[0])
            return lat, lon
        except:
            return None
    
    def network_sleep(self, flag, delay) -> None:
        if flag:
            time.sleep(delay)

    def try_parse_by_one(self, idx, pattern, parsing_func, exist_check, stage_delay, matrix, urls, network_busy):
        """ Попытаться распарсить url соответствующим методом """
        if exist_check[pattern]:
            network_busy |= True
            cur_urls = np.array(urls)[matrix[idx]].tolist()
            for cur_url in cur_urls:
                res = parsing_func(cur_url)
                if res is not None:
                    return res, exist_check, pattern
                
        self.network_sleep(network_busy, stage_delay)
        return network_busy

    def process(self, addr, num_results=10, google_delay=2, stage_delay=2, proxy=None, banned=False, captcha_sleep=2):
        """
        1. Если есть alta, то альта
            если она none, то freicon если есть
                если none, то yandex
                    если none то 2gis => далее none тк none
        2. Если нет alta, есть freicon, то freicon
            если none, то yandex
                если none то 2 gis => далее none тк none
        3. если нет ни alta ни freicon, то yandex
            если none то 2gis => далее none тк none

        :return: returned value must be in ['banned', 'nan', (lat, lon)]
        """
        self.patterns = ['yandex.ru', '2gis.ru', 'alta.ru', 'freicon.ru']
        self.funcs = [self.parse_yandex, self.parse_2gis, self.parse_alta, self.parse_freicon]

        try:
            urls = self.google_address(addr=addr, num_results=num_results, proxy=proxy, banned=banned, captcha_sleep=captcha_sleep)
            matrix = np.array([[pattern in url.lower() for url in urls] for pattern in self.patterns])

            if (len(urls) < 2) or (matrix.sum() < 1):
                time.sleep(google_delay)
                
                urls = self.google_address(addr=addr[:int(len(addr)/2)], num_results=num_results, proxy=proxy, banned=banned, captcha_sleep=captcha_sleep)
                matrix = np.array([[pattern in url.lower() for url in urls] for pattern in self.patterns])
                if (matrix.sum() < 1):
                    time.sleep(google_delay)
                    urls = self.google_address(addr=addr[int(len(addr)/2):], num_results=num_results, proxy=proxy, banned=banned, captcha_sleep=captcha_sleep)
                    matrix = np.array([[pattern in url.lower() for url in urls] for pattern in self.patterns])
                    if (matrix.sum() < 1):
                        time.sleep(google_delay)
                        sp = addr.split()
                        urls = self.google_address(addr=' '.join(sp[2:len(sp)-1]), num_results=num_results, proxy=proxy, banned=banned, captcha_sleep=captcha_sleep)
                        matrix = np.array([[pattern in url.lower() for url in urls] for pattern in self.patterns])
                        if (matrix.sum() < 1):
                            self.matrix = matrix
                            self.urls = urls
                            return 'nan'
        except Exception as e:
            print(f'Probably, banned! {e}')
            return 'banned'

        exist_check = {k:v for k, v in zip(self.patterns, matrix.sum(axis=1).tolist())}
        network_busy = False
        list_enumerate = list(enumerate(zip(self.patterns, self.funcs)))

        # shuffle `list_enumerate` in right order and execute
        for idx, (pattern, parsing_func) in list_enumerate[2:-1] + list_enumerate[:2] + [list_enumerate[-1]]:
            result = self.try_parse_by_one(idx, pattern, parsing_func, exist_check, stage_delay, matrix, urls, network_busy)
            if isinstance(result, tuple):
                return result
            network_busy = result

        self.matrix = matrix
        self.urls = urls
        return 'nan'
    
    def process_v2(self, addr_iter):
        ya_home_url = 'https://yandex.ru/maps'
        browser = wd.Chrome(options=self.options)
        browser.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        browser.get(ya_home_url)
        time.sleep(10)
        try:
            results = []
            for addr in addr_iter:
                elem = browser.find_element(By.CSS_SELECTOR, 'input[placeholder="Поиск мест и адресов"]')
                elem.send_keys(addr)
                # for char in addr:
                #     elem.send_keys(char)
                #     time.sleep(random.uniform(0.1, 0.5))  # Случайная задержка

                elem.send_keys(Keys.ENTER)

                # Ожидаем загрузку результатов с рандомной задержкой
                time.sleep(random.uniform(1, 4))

                # Получаем HTML код страницы
                html = browser.page_source
                soup = BeautifulSoup(html, "html.parser")

                # Парсим результат
                result_v1 = self.parse_yandex(soup=soup)
                if result_v1 is None:
                    print('Parsing v2 and nan')
                    try:
                        route = browser.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[10]/div[1]/div[1]/div[1]/div/div[1]/div/div/div[3]/div[3]/div/div[7]/div/div/div/div/div/div[1]/div/button')
                        route.click()
                    except KeyboardInterrupt:
                        return results
                    except:
                        try:
                            route = browser.find_element(By.CSS_SELECTOR, 'input[placeholder="Поиск мест и адресов"]')
                            route.click()
                            time.sleep(random.uniform(0.1, 0.3))
                            html = browser.page_source
                            soup = BeautifulSoup(html, "html.parser")
                            s = soup.find('li', class_='search-snippet-view')
                            divs = s.findAll('div')
                            for div in divs:
                                if div.has_attr('data-coordinates'):
                                    results.append(','.join(div['data-coordinates'].split(',')[::-1]))
                                break
                            # time.sleep(random.uniform(1, 3))
                            try:
                                clear_button = browser.find_element(By.CSS_SELECTOR, "button[aria-label='Закрыть']")
                                clear_button.click()
                                print('go back')
                            except KeyboardInterrupt:
                                return results
                            except:
                                print('go back failed')
                        except KeyboardInterrupt:
                            return results
                        except:
                            """ возможно вышел лист адресов """
                            try:
                                html = browser.page_source
                                soup = BeautifulSoup(html, "html.parser")
                                s = soup.find('li', class_='search-snippet-view')
                                divs = s.findAll('div')
                                for div in divs:
                                    if div.has_attr('data-coordinates'):
                                        results.append(','.join(div['data-coordinates'].split(',')[::-1]))
                                    break
                                print('closing addr')
                                # time.sleep(random.uniform(0.5, 4))
                                clear_button = browser.find_element(By.CSS_SELECTOR, "button[aria-label='Закрыть']")
                                clear_button.click()
                                # time.sleep(random.uniform(0.5, 4))
                            except:
                                print(f'Не кликнулось, {addr=}')
                                try:
                                    clear_button = browser.find_element(By.CSS_SELECTOR, "button[aria-label='Закрыть']")
                                    clear_button.click()
                                except:
                                    print('WARNING: НЕ СРАБОТАЛО ЗАКРЫТИЕ')
                                results.append('nan')
                        
                else:
                    # time.sleep(random.uniform(0.5, 4))
                    clear_button = browser.find_element(By.CSS_SELECTOR, "button[aria-label='Закрыть']")
                    clear_button.click()
                    # time.sleep(random.uniform(0.5, 4))
                    results.append(result_v1)
                print(f'{results=}')
                time.sleep(random.uniform(1, 2.5))
        except KeyboardInterrupt:
            return results
        return results
        

geopc = GeoProcessor()

## testing

In [74]:
geopc.distance(coordinates[0], tgt_coord)

distance using OSRM:  1040.1744


1040.1744

In [75]:
query = next(buyer_address_iter)
query

'Урманское месторождение '

In [76]:
geopc.process(query, banned=True)

'nan'

In [134]:
results = geopc.process_v2(buyer_address_iter)

results=[(61.166542, 75.967595)]
Parsing v2 and nan
go back
results=[(61.166542, 75.967595), '68.486525,73.557826']
results=[(61.166542, 75.967595), '68.486525,73.557826', (59.942462, 30.370567)]
results=[(61.166542, 75.967595), '68.486525,73.557826', (59.942462, 30.370567), (55.654198, 37.55602)]
Parsing v2 and nan
Не кликнулось, addr='ст. Лабытнанги, Северной ж.д., код 289904 '
results=[(61.166542, 75.967595), '68.486525,73.557826', (59.942462, 30.370567), (55.654198, 37.55602), 'nan']
Parsing v2 and nan
go back
results=[(61.166542, 75.967595), '68.486525,73.557826', (59.942462, 30.370567), (55.654198, 37.55602), 'nan', '66.024437,57.547413']
Parsing v2 and nan
Не кликнулось, addr='Регион Центр '
results=[(61.166542, 75.967595), '68.486525,73.557826', (59.942462, 30.370567), (55.654198, 37.55602), 'nan', '66.024437,57.547413', 'nan']
results=[(61.166542, 75.967595), '68.486525,73.557826', (59.942462, 30.370567), (55.654198, 37.55602), 'nan', '66.024437,57.547413', 'nan', (55.074113, 

In [138]:
sum([i == 'nan' for i in results]), len(results), (len(results) - sum([i == 'nan' for i in results]))/len(results)

(13, 116, 0.8879310344827587)

In [132]:
l = [(59.942462, 30.370567), 'nan', '51.131434,61.861566', (56.77668, 105.708721), (56.107614, 38.845192), (56.592407, 57.769856), (56.842551, 35.759937), 'nan', '60.952959,76.608819', (56.319479, 43.916523), (48.653916, 44.446368), (55.252347, 61.290138), 'nan', 'nan', 'nan', (69.028365, 33.075304), '66.522781,66.586467', (57.153214, 65.564511), (59.8165, 30.294057), (57.152384, 65.567152), 'nan', 'nan', (55.643896, 37.796939), (57.151652, 65.53342)]
dt = 140 / len(l)
f"{dt:.2f} секунды на адрес; ", f"{dt*1600/3600:.2f} часа на датасет; ", f"{dt*120} секунд для валидации алгоритма"

('5.83 секунды на адрес; ', '2.59 часа на датасет')

In [109]:
s = BeautifulSoup('<li class="search-snippet-view"><div class="search-snippet-view__body _type_toponym" data-coordinates="74.542409,63.794844" data-id="1639169560" data-log-id="dHlwZT1nZW9jb2RlcjthZGRyZXNzPWdlb2NvZGVyMQ==" data-object="search-list-item"><div aria-hidden="false" role="button" tabindex="0"><div class="search-snippet-view__content"><div class="search-snippet-view__head"><div class="search-snippet-view__title">Панель 11</div></div><div class="search-snippet-view__description">Ямало-Ненецкий автономный округ, Муравленко, Панель 11</div></div></div></div></li>', parser='html.parser')
divs = s.findAll('div')
for div in divs:
    if div.has_attr('data-coordinates'):
        print(','.join(div['data-coordinates'].split(',')[::-1]))
    break

63.794844,74.542409


# Processing data

In [13]:
# 193 дубликата адреса
buyer_address = pd.DataFrame(buyer.address.value_counts()).reset_index()
buyer_address_iter = iter(buyer_address.address.values)
"{}/{} duplicates. Percentage of duplicates: {}. Number of unique companies: {}".format(dup:=buyer_address['count'].apply(lambda x: x - 1 if x > 1 else 0).sum(), buyer.shape[0], f'{dup/buyer.shape[0]*100:.2f}%', buyer_address.shape[0])

'193/1856 duplicates. Percentage of duplicates: 10.40%. Number of unique companies: 1663'

result:
```
((lat, lon),
 {'yandex.ru': 0, '2gis.ru': 0, 'alta.ru': 1, 'freicon.ru': 1},
 'alta.ru')
```
 or

 'nan'

In [14]:
def prepare_result(result):
    if isinstance(result, tuple):
        return list(result[0]) + list(result[1].values()) + [result[2]]
    else:
        return [result]*7

In [16]:
import sys
save_name = '../table_coords.csv'
table = None
proxy = None

if Path(save_name).exists():
    table = pd.read_csv(save_name)
    need_addr = np.array(sorted(set(buyer_address.address.values.tolist()) - set(table.addr.values.tolist())))
else:
    need_addr = buyer_address.address.values

banned_timeout_iterations = 80

requests_banned_flag = True
num_captcha = 0
captcha_timeout = 150
for cur_idx, address in tqdm(enumerate(iter(need_addr)), total=need_addr.shape[0]):
    if (not requests_banned_flag) or (cur_banned_timeout > banned_timeout_iterations):
        result = prepare_result(geopc.process(address, google_delay=2, stage_delay=2))
        requests_banned_flag = 'banned' in result
        cur_banned_timeout = 0
    
    if requests_banned_flag:
        captcha_sleep = 2 if num_captcha < captcha_timeout - 2 else 45
        result = prepare_result(geopc.process(address, google_delay=2, stage_delay=2, proxy=proxy, banned=True, captcha_sleep=captcha_sleep))
        cur_banned_timeout += 1
        print(f'{cur_banned_timeout=}/{banned_timeout_iterations}')
        if 'banned' in result:
            sys.exit()
        num_captcha += 1
        if num_captcha > captcha_timeout + 1:
            num_captcha = 0
    # time.sleep(20)
    if table is None:
        table = pd.DataFrame({'addr': address, 'lat': [result[0]], 'lon': [result[1]], 'yandex': [result[2]], '2gis': [result[3]], 'alta': [result[4]], 'freicon': [result[5]], 'executed': [result[6]]})
    else:
        tmp = pd.DataFrame({'addr': address, 'lat': [result[0]], 'lon': [result[1]], 'yandex': [result[2]], '2gis': [result[3]], 'alta': [result[4]], 'freicon': [result[5]], 'executed': [result[6]]})
        table = pd.concat([table, tmp]).reset_index(drop=True)
    table.to_csv(save_name, index=False, encoding='UTF-8')
    time.sleep(3)

  0%|          | 0/1494 [00:00<?, ?it/s]

cur_banned_timeout=1/80


  0%|          | 1/1494 [00:13<5:40:31, 13.68s/it]

cur_banned_timeout=2/80


  0%|          | 2/1494 [00:29<6:07:51, 14.79s/it]

...

cur_banned_timeout=37/80


 10%|▉         | 145/1494 [42:03<10:01:10, 26.74s/it]

cur_banned_timeout=38/80


 10%|▉         | 146/1494 [42:59<6:36:59, 17.67s/it] 


KeyboardInterrupt: 

In [127]:
address

'109429, г. Москва, р-н Капотня, д. 38, стр. 1 '

In [128]:
geopc.process(address, google_delay=2, stage_delay=2, proxy=proxy, banned=True)

((55.639613, 37.810018),
 {'yandex.ru': 2, '2gis.ru': 1, 'alta.ru': 0, 'freicon.ru': 0},
 'yandex.ru')

In [29]:
geopc.process(address)

'nan'

In [30]:
geopc.urls

[]

In [15]:
geopc.google_address(address)

HTTPError: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3D%25D1%2581%25D1%2582.%2B%25D0%259F%25D1%2580%25D0%25B8%25D0%25BE%25D0%25B1%25D1%258C%25D0%25B5%2B%25D0%25A1%25D0%25B2%25D0%25B5%25D1%2580%25D0%25B4%25D0%25BB%25D0%25BE%25D0%25B2%25D1%2581%25D0%25BA%25D0%25BE%25D0%25B9%2B%25D0%25B6%252F%25D0%25B4%252C%2B%25D0%25BA%25D0%25BE%25D0%25B4%2B779409%252C%2B%25D0%25BA%25D0%25BE%25D0%25B4%2B%25D0%25BF%25D0%25BE%25D0%25BB%25D1%2583%25D1%2587%25D0%25B0%25D1%2582%25D0%25B5%25D0%25BB%25D1%258F%2B6162%2B%26num%3D7%26hl%3Den%26start%3D0%26safe%3Dactive&hl=en&q=EgRdrw1rGOXllrgGIjAZcID5KfPfBJgguDMlQepjKhzx2OF38kHRZIoQB_kQ3myuKfc_M-jTo1CJN46oATkyAXJaAUM

In [1]:
import pandas as pd

In [3]:
pd.read_csv('/home/yaroslav/progs/digitdep_hack/table_coords.csv').addr.values[-1]

'109429, г. Москва, р-н Капотня, кварт. 2, д. 20а '

In [9]:
class SearchResult:
    def __init__(self, url, title, description):
        self.url = url
        self.title = title
        self.description = description

    def __repr__(self):
        return f"SearchResult(url={self.url}, title={self.title}, description={self.description})"

In [16]:
address = '109429, г. Москва, р-н Капотня, кв-л 2, д.1, корп.3 '

In [23]:
from copy import deepcopy
options = deepcopy(geopc.options)
options.page_load_strategy = 'eager'

browser = wd.Chrome(options=options)
browser.get('https://www.google.com/search')
textarea = browser.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea")
textarea.send_keys(address)
textarea.send_keys(Keys.ENTER)
time.sleep(1)
html = browser.page_source

In [116]:
soup = BeautifulSoup(html, "html.parser")

In [117]:
list(parse_soup(soup))

['https://yandex.ru/maps/213/moscow/house/2_y_kvartal_1k3/Z04YcQ5mQEYOQFtvfXp1cnRqYg==/',
 'https://2gis.ru/moscow/geo/4504235282869830',
 'https://yandex.ru/maps/213/moscow/house/1_y_kvartal_3/Z04YcQ5gS00FQFtvfXp1cHprYg==/',
 'http://mosopen.ru/street/9443',
 'https://www.pochta.ru/indexes/4efb6d0a-abfd-443a-9c56-743f77959abe']